---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV


def blight_model():
    
    # Let´s start reading the csv file, we need a special encoding for this
    train_df = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    train_df.index = train_df['ticket_id']    
    # DATA CLEANING
    ## train_df.describe(include='all')
    
    # Now we will clean the df a little bit, first by removing from the train dataset those features which won't be available during the test
    train_df = train_df.drop(['ticket_id', 'payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 'compliance_detail'], axis=1)
    
           
    # We will also remove the violation_zip_code, non_us_str_code and grafitti_status because they are only filled with NaN
    ##train_df.isnull().sum(axis = 0) <-- This shows us the number of NaNs per column
    train_df.drop(['violation_zip_code', 'non_us_str_code', 'grafitti_status'], axis=1, inplace = True)
    
       
    # Now we convert the NaN from the target value column which we need to predict, the compliance,which is True (1) if the ticket was paid early, on time, or within one month of the hearing data, False (0) the ticket was paid after the hearing date or not at all, and NaN (-1)
    # if violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.
    # All tickets where the violators were found not responsible are not considered during evaluation and therefore we will remove them to train the algorithm. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.
    
    # We change the NaN for -1
    train_df.compliance = train_df.compliance.fillna(-1)
    
    # We remove the rows where compliance is -1 because those will not be considered during evaluation, as explained before    
    train_df = train_df[train_df.compliance != -1.0]
    
    
    
    
    # MACHINE LEARNING
    
    # We check the correlation between all the columns and the compliance:
    ## train_df.corr()
    # Thanks to this we see that violation_street_number, late_fee and discount_amount could be good features to start with
    final_train_df = train_df[['violation_street_number', 'late_fee', 'discount_amount', 'compliance']]
    
    # We split the dataset, assigning X to the features column and y to the target column
    X = final_train_df.iloc[:,0:-1]
    y = final_train_df.compliance
    

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

    # We use GridSearchCV to obtain the best parameters for a random forest classifier
    
    ##model = RandomForestClassifier()
    ##clf.fit(X_train, y_train)
    ##train_acc = clf.score(X_train, y_train)
    ##test_acc = clf.score(X_test, y_test)
    
    ##param_grid = {'n_estimators':[7,8,9,10], 'max_depth':[8,9,10]}
    ##grid_search = GridSearchCV(model, param_grid, cv =10, scoring="roc_auc") <- We use the roc_auc as scoring metric
    ##grid_result = grid_search.fit(X_train, y_train)
    # summarize results
    ##print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    
    ## From here we get that the best AUC is 0.766081 using {'max_depth': 10, 'n_estimators': 7}
    
    
    
    clf = RandomForestClassifier(n_estimators = 7, max_depth = 10)
    clf.fit(X_train, y_train)
       
    # Probabilities
    test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    test_df.index = test_df['ticket_id']
    X_predict = clf.predict_proba(test_df[['violation_street_number', 'late_fee', 'discount_amount']])
    ## Alternatively, uncommenting the GridSearchCV part and then in X_predict instead of clf use grid_result, but it takes longer to run
    
    # Confusion Matrix
    ##clf_predicted = clf.predict(X_test)
    ##confusion = confusion_matrix(y_test, clf_predicted)
    
    # predict_proba gives an output array of two elements, the first one is the probabilitz that the target value is 0/False (in this case that the person will not pay)
    # and the other one the probabilitz of 1/True (in this case that the person will pay). This is why we need to select just the second element, because the exercise rtequests
    # to predict the prob of the person paying
    ans = pd.Series(data = X_predict[:,1], index = test_df['ticket_id'], dtype='float32')
    
    return ans


blight_model()


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Best: 0.766073 using {'max_depth': 9, 'n_estimators': 10}


ticket_id
284932    0.057185
285362    0.033191
285361    0.060806
285338    0.062961
285346    0.069829
285345    0.057933
285347    0.065780
285342    0.238427
285530    0.024967
284989    0.021740
285344    0.065780
285343    0.026091
285340    0.026091
285341    0.065780
285349    0.069829
285348    0.057933
284991    0.023909
285532    0.032395
285406    0.031166
285001    0.035821
285006    0.026091
285405    0.029599
285337    0.035818
285496    0.049210
285497    0.089894
285378    0.042552
285589    0.033019
285585    0.059776
285501    0.067324
285581    0.033091
            ...   
376367    0.021890
376366    0.034710
376362    0.032945
376363    0.057048
376365    0.021890
376364    0.034710
376228    0.041214
376265    0.028653
376286    0.309529
376320    0.038023
376314    0.023751
376327    0.435414
376385    0.318512
376435    0.356396
376370    0.356396
376434    0.044800
376459    0.070341
376478    0.014631
376473    0.034710
376484    0.028848
376482    0.033019
37

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


KeyboardInterrupt: 